In [ ]:
data = {
    'xlsx'  : '/mnt/d/autocls_e3/classref/refexp32_vec.shp',
    'sheet' : None,
    'pxid'  : 'pxid',
    'classcol' : 'rdv_class'
}

yescls = 'roads'
nocls = 'no_roads'
mixcls = []

outshp = '/mnt/d/autocls_e3/trainref/exp33_vec.shp'
outrst = '/mnt/d/autocls_e3/trainref/exp33_rst.tif'

refrst = '/mnt/d/autocls_e3/stdlmt/stdrst_valeflores.tif'

expname = 'exp33'

db = 'gthruth_valeflores_v3'

In [ ]:
import json as js
import random as rnd
import numpy as np
import pandas as pd
from glass.rd import tbl_to_obj
from glass.pd.stats import df_to_freqdf
from glass.it.shp import dbtbl_to_shp
from glass.prop.prj import get_epsg
from glass.dtt.rst.torst import shp_to_rst

In [ ]:
epsg = get_epsg(refrst)

In [ ]:
if is_shp(data['xlsx']):
    ddf = shp_to_obj(data['xlsx'])
else:
    ddf = tbl_to_obj(data['xlsx'], sheet=data['sheet'])

In [ ]:
# Count number of classes
ddf['classes'] = ddf['classes'].str.replace("'", '"')
ddf['classes'] = ddf['classes'].apply(js.loads)
ddf['nclasses'] = ddf['classes'].map(len)

In [ ]:
ddf['subcls'] = np.where(
    ddf[data['classcol']].isin(mixcls),
    '-+1', ''
)

In [ ]:
ddf['subcls'] = np.where(
    (ddf[data['classcol']] == yescls) & (ddf.nclasses > 1),
    '1+', ddf.subcls
)

ddf['subcls'] = np.where(
    (ddf[data['classcol']] == nocls) & (ddf.nclasses > 1),
    '1-', ddf.subcls
)

ddf['subcls'] = np.where(
    (ddf[data['classcol']] == yescls) & (ddf.nclasses == 1),
    '+' + ddf['classmax'], ddf.subcls
)

ddf['subcls'] = np.where(
    (ddf[data['classcol']] == nocls) & (ddf.nclasses == 1),
    '-' + ddf['classmax'], ddf.subcls
)

In [ ]:
# Give random numbers to each row
ddf['rnd'] = ddf.apply(lambda _: rnd.randint(1, 1000), axis=1)

# Scramble
ddf= ddf.sort_values(by=['subcls', 'rnd'])
ddf.reset_index(inplace=True)

In [ ]:
ddf

In [ ]:
ddf['cumaux'] = 1

ddf['cum'] = ddf.groupby('subcls')['cumaux'].transform(pd.Series.cumsum)
ddf['grpmax'] = ddf.groupby('subcls')['cum'].transform(pd.Series.max)

ddf['relcum'] = ddf.cum / ddf.grpmax * 100

In [ ]:
train_df = ddf[ddf.relcum <= 30]

In [ ]:
# Train to SHAPE

whr = ", ".join(train_df.rootid.astype(str).tolist())

q = (
    "SELECT jtbl.*, mtbl.geom "
    "FROM gthruth_fishnet AS mtbl "
    "LEFT JOIN ("
    	"SELECT mm.pxid, jj.* "
    	"FROM pxexpclasses AS mm "
    	"INNER JOIN ("
    		"SELECT mj.expclsid, mj.codeval, mj.nameval, "
    		"jt.slug AS expslug "
    		"FROM expclasses AS mj "
    		"INNER JOIN experiments AS jt "
    		"ON mj.expfk = jt.eid "
            f"WHERE jt.slug = '{expname}'"
    	") AS jj "
    	"ON mm.eclsid = jj.expclsid"
    ") AS jtbl "
    "ON mtbl.cid = jtbl.pxid "
    f"WHERE mtbl.cid = ANY(ARRAY[{whr}])"
)

dbtbl_to_shp(
    db, q, "geom", outshp,
    tableIsQuery=True, epsg=epsg
)

In [ ]:
# Train to raster

shp_to_rst(
    outshp, 'codeval', None, 0,
    outrst, rst_template=refrst,
    api='gdal', rtype=int, dtype='UInt16'
)